# Description
Notebook contains full implementation of the FFT-based model estimation. Sample FX data is used for an example.
Estimation implemented here covers GBM, VG, CGMY and $t$-distribution model.

# 1 Imports
Package `lmfit` is used for the minimization. Check https://lmfit.github.io/lmfit-py/ for more details.
If using `Anaconda`, install `lmfit` by running `conda install lmfit` in your command line.

In [30]:
import pandas as pd
import numpy as np
from scipy.fft import fft
from lmfit import minimize, Minimizer, Parameters
import warnings
warnings.filterwarnings('ignore')

# 2 Load FX data and compute log returns

In [31]:
data = pd.read_excel('data_FX.xlsx')
data['prev_price'] = data['price'].shift(1)
data['logret'] = data.eval('log(price/prev_price)')

# 3 Definition of the Estimator object

In [32]:
class Estimator:
    '''
        Object Estimator provides estimate of a stochatic model using FFT method
    '''
    N    = 2**16
    eta  = 0.5
    
    dt   = 1/252
    
    def __init__(self, data_returns, N = None, eta = None, dt = None):
        '''
            Data returns: array-like of returns. NaNs will be removed
            N           : (optional) number of integration points in the u-space
            eta         : (optional) eta = delta u, i.e. step size of the integration
            dt          : (optional) dt = h; time steps. If log returns are daily log returns then it is suggested to use h = 1/252
            
        '''
        self.data_returns = pd.Series(data_returns).dropna()
        
        if N is not None:
            self.N = N
        
        if eta is not None:
            self.eta = eta
        
        if dt is not None:
            self.dt  = dt
            
        self.make_fft_params()
    
    def make_fft_params(self):
        '''
            Function pepares FFT and integration variables
        '''
        self.b    = np.pi/self.eta
        self.lbda = 2*np.pi/(self.eta * self.N)
        
        j         = np.arange(0, self.N, dtype = np.float64)
        self.u    = j * self.eta
        self.x    = -self.b + j * self.lbda   
        
        w         = np.ones_like(j)
        w[0]      = 0.5
        w[-1]     = 0.5
        self.w    = w
    
    def get_cfXh(self, model):
        '''
            Function returns CF for X_h, i.e. CF for the random component in the model
            
            User is free to add CF for another model to the if statement below
        '''
        
        if model.lower() == 'gbm':
            return lambda params, u: np.exp(-0.5* params['sigma']**2 *self.dt* u**2)
        elif model.lower() == 'vg':
            return lambda params, u: (1- 1j*u*params['theta']*params['nu'] + 0.5 * params['sigma']**2 * u**2 * params['nu'])**(-self.dt / params['nu'])
        elif model.lower() == 'cgmy':
            from scipy.special import gamma
            return lambda params, u: np.exp(params['C']* self.dt * gamma(-params['Y'])* ((params['M'] - 1j*u)**params['Y'] - params['M']**params['Y'] + (params['G']+1j*u)**params['Y'] - params['G']**params['Y']) )
        elif model.lower() == 't':
            from scipy.special import gamma, kv
            modu = lambda params, u: np.abs(params['sigma'] * np.sqrt(self.dt * (params['nu']-2.0)) * u)
            return lambda params, u: (kv(params['nu']/2.0, modu(params, u)) * modu(params, u) ** (params['nu']/2))/(gamma(params['nu']/2.0) * 2.0**(params['nu']/2.0-1.0))
        else:
            return None
        
    def get_init_estim_params(self, model):
        '''
            Function returns 'initial guess' parameters for a given model
            For exact details about how to specify the parameters, read: https://lmfit.github.io/lmfit-py/parameters.html
        '''
        
        params = Parameters()
        
        if model.lower() == 'gbm':
            params.add_many(('mu',    0.01, True, None,     None, None, None),
                            ('sigma', 0.10, True, 0.00001,  None, None, None))
        elif model.lower() == 'vg':
            params.add_many(('mu',    0.01, True, None,     None, None, None),
                            ('sigma', 0.10, True, 0.00001,  None, None, None),
                            ('theta', 0.00, True, None,     None, None, None),
                            ('nu',    0.01, True, 0.00001,  None, None, None))
        elif model.lower() == 'cgmy':
            params.add_many(('mu',    0.01, True, None,     None, None, None),
                            ('C',     0.01, True, 0.00001,  None, None, None),
                            ('G',     20,   True, 0.00001,  None, None, None),
                            ('M',     20,   True, 0.00001,  None, None, None),
                            ('Y',     1.85, True, None,     2.0, None, None))
        elif model.lower() == 't':
            params.add_many(('mu',    0.01, True, None,     None, None, None),
                            ('sigma', 0.1,  True, 0.00001,  None, None, None),
                            ('nu',    100,  True, 2.00001,  None, None, None))
            self.u = np.clip(self.u, a_min = 0.001, a_max = None) #small override because CF evaluated at u=0 gives NaNs
        return params
    
    def opt_fun(self, params, model):
        '''
            Implementation of MLE -LL(params) objective function (defined by means of the FFT) that will be minimized
        '''
        paramsdict = params.valuesdict()
        u          = self.u
        cfXh       = self.get_cfXh(model)
        omega      = -1/self.dt * np.log(cfXh(paramsdict, -1j))
        cf         = np.exp(1j*u * (paramsdict['mu']*self.dt + omega*self.dt))*cfXh(paramsdict, u)
        
        densities_fft = 1/np.pi * fft(np.exp(1j*self.b*u) * cf * self.eta * self.w)
        densities     = np.clip(np.interp(self.data_returns, self.x, np.real(densities_fft)), a_min = 0.000000001, a_max = None)

        return -np.sum(np.log(densities))
    
    def estimate(self, model, method = 'nelder'):
        '''
            Function execute the model fitting. By default it uses Nelder-Mead algorithm
            
            It uses lmfit package to find the optimal parameters minimizing -LL(params) funtion
        '''
        print(f'Estimating parameters of a {model} model')
        params    = self.get_init_estim_params(model)
        minimizer = Minimizer(self.opt_fun, params, fcn_args = (model,), nan_policy = 'omit')
        minimizer.minimize(method = method)
        return minimizer.result

# 4 Estimation

## 4.1 Estimate GBM model

In [33]:
Estimator(data['logret']).estimate('GBM')

Estimating parameters of a GBM model


## 4.2 Estimate Variance Gamma model

In [34]:
Estimator(data['logret']).estimate('VG')

Estimating parameters of a VG model


## 4.3 Estimate CGMY model

In [35]:
Estimator(data['logret']).estimate('CGMY')

Estimating parameters of a CGMY model


## 4.4 $t$-distribution model

In [36]:
Estimator(data['logret']).estimate('t')

Estimating parameters of a t model
